In [1]:
# !pip install pandas findspark py4j seaborn numpy
# !pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio===0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip install transformers
# !pip install tqdm
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [2]:
# use if the above installing of torch breaks
# !pip install torch torchvision torchaudio


In [3]:
#Doesn't seem to work here properly

import warnings
warnings.filterwarnings('ignore')

warnings.filterwarnings(action='once')

In [4]:
from tqdm.notebook import tqdm

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

In [6]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pkg_resources
pkg_resources.require("torch==1.11.0")
import torch

import pandas as pd
import numpy as np

from time import sleep
import json
import os

from collections import namedtuple
import sqlite3

In [8]:
pd.set_option('display.max_colwidth', None)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
import findspark

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [10]:
findspark.init('/home/venky/spark-3.2.1-bin-hadoop3.2')
findspark.add_packages("org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")

In [11]:
spark = SparkSession.builder.appName("FYP").getOrCreate()
spark

In [12]:
#Run this only once, restart kernel if errors
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=FYP>

In [13]:
TABLE_COUNT = 0
INMEMORY_TABLENAME = "StockData"
SQLITE_TABLENAME = "scored_stocks"
OFFSET = 0
TOPIC = "gaming-stocks"

In [14]:
connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/cache.sqlite'))
cursor = connection.cursor()

query = f"SELECT offsetval FROM OFFSET_FINDER WHERE topic LIKE ?"

rows = cursor.execute(query, [TOPIC]).fetchall()

if rows:
    OFFSET = rows[0][0]
else:
    insert_query = f"INSERT INTO OFFSET_FINDER VALUES(?, ?)"
    cursor.execute(insert_query, (TOPIC, 0))
    connection.commit()

print({f"Starting Offset for {TOPIC}": OFFSET})
    
cursor.close()
connection.close()

{'Starting Offset for gaming-stocks': 501}


In [15]:
# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribe", TOPIC) \
#   .option("startingOffsets", f""" {{"{TOPIC}":{{"0":{OFFSET}}}}} """) \
#   .load()

# schema_str = "Data STRING"

# df = df.selectExpr("CAST(value AS STRING)")
# df = df.select(from_csv(col("value"),schema_str).alias("Table"))
# df = df.selectExpr("Table.*")
# df.printSchema()
# #option("truncate", "false")

In [16]:
# query = df.writeStream.trigger(processingTime='5 seconds').queryName(f"{INMEMORY_TABLENAME}{TABLE_COUNT}").format('memory').outputMode("append").start()


In [17]:
spark.sql('SHOW TABLES').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [18]:
def my_round(val):
    arr = val.split('.')
    dec = ""
    if len(arr) > 1:
        dec = arr[1][:3]
    val = f"{arr[0]}.{dec}"
    return float(val)

def my_round1(Open, Close):
    val = str(100*((Close-Open)/Open))
    return my_round(val)

In [19]:
# sleep(10)

# tweet_dict_list = []

# value = spark.sql(f"SELECT * FROM {INMEMORY_TABLENAME}{TABLE_COUNT} LIMIT 10").collect()

# for row in value:
#     #print(row)
#     jsonCopy = json.loads(row["Data"])
#     jsonCopy['open'] = float(jsonCopy['open'])
#     jsonCopy['close'] = float(jsonCopy['close'])
#     jsonCopy['percentage'] = my_round1(jsonCopy['open'], jsonCopy['close'])
#     tweet_dict_list.append(jsonCopy)
# pdd = pd.DataFrame(tweet_dict_list)

# query.awaitTermination(1)
# pdd.head(10)

In [20]:
# rdd = sc.parallelize(tweet_dict_list)

In [21]:
# %%time
# rdd.map(lambda row: (row['stockDate'], row['ticker'] , row['percentage'])).toDF().toPandas().head(10)

In [22]:
# newrdd = rdd.map(lambda row: (row['category'], row['stockDate'], row['percentage']))
# newrdd.collect()

In [23]:
# nextrdd = newrdd.map(lambda tup: ((tup[0], tup[1]), tup[2])).reduceByKey(lambda a, b: (a+b)/2).map(lambda row: (row[0], row[1]))
# nextrdd = newrdd.map(lambda tup: ((tup[0], tup[1]), (tup[2], 1))).reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])).map(lambda row: (row[0], row[1][0]/row[1][1]))
# nextrdd.collect()

In [24]:
# #write to db
# connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/results.sqlite'))
# cursor = connection.cursor()

# drop_table = f'''
#             DROP TABLE IF EXISTS {SQLITE_TABLENAME};
#             '''

# cursor.execute(drop_table)


# create_table = f'''CREATE TABLE IF NOT EXISTS {SQLITE_TABLENAME} (
#                 category TEXT,
#                 date DATE,
#                 agg_percent TEXT,
#                 CONSTRAINT uniq_stock PRIMARY KEY (category, date)
#                 );
#                 '''

# cursor.execute(create_table)

# insert_records = f'''INSERT INTO {SQLITE_TABLENAME} (category, date, agg_percent) VALUES(?, ?, ?)
#                         ON CONFLICT(category, date) DO 
#                         UPDATE SET agg_percent = (agg_percent + excluded.agg_percent)/2
#                         WHERE {SQLITE_TABLENAME}.category LIKE ? AND {SQLITE_TABLENAME}.date LIKE ? '''
    

# contents = []
# for row in nextrdd.collect():
#     contents.append((row[0][0], row[0][1], row[1], row[0][0], row[0][1]))
    
# try:
#     cursor.executemany(insert_records, contents)
#     connection.commit()

#     rows = cursor.execute(f"SELECT * FROM {SQLITE_TABLENAME}").fetchall()
#     for row in rows:
#         print(row)
# except sqlite3.Error as error:
#     print({error})
# finally:
#     cursor.close()
#     connection.close()

In [25]:
def init_df_table():
    df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", TOPIC) \
      .option("startingOffsets", f""" {{"{TOPIC}":{{"0":{OFFSET}}}}} """) \
      .load()

    schema_str = "Data STRING"

    df = df.selectExpr("CAST(value AS STRING)")
    df = df.select(from_csv(col("value"),schema_str).alias("Table"))
    df = df.selectExpr("Table.*")
    df.printSchema()

    query = df.writeStream.trigger(processingTime='5 seconds').queryName(f"{INMEMORY_TABLENAME}{TABLE_COUNT}").format('memory').outputMode("append").start()
    spark.sql('SHOW TABLES').show()
    return query

def delete_test_table():
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/results.sqlite'))
    cursor = connection.cursor()

    drop_table = f'''
            DROP TABLE IF EXISTS {SQLITE_TABLENAME};
            '''

    cursor.execute(drop_table)


    create_table = f'''CREATE TABLE IF NOT EXISTS {SQLITE_TABLENAME} (
                category TEXT,
                date DATE,
                agg_percent TEXT,
                CONSTRAINT uniq_stock PRIMARY KEY (category, date)
                );
                '''


    cursor.execute(create_table)
    
    cursor.close()
    connection.close()

def write_to_db(rdd):
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/results.sqlite'))
    cursor = connection.cursor()
    
    insert_records = f'''INSERT INTO {SQLITE_TABLENAME} (category, date, agg_percent) VALUES(?, ?, ?)
                        ON CONFLICT(category, date) DO 
                        UPDATE SET agg_percent = (agg_percent + excluded.agg_percent)/2
                        WHERE {SQLITE_TABLENAME}.category LIKE ? AND {SQLITE_TABLENAME}.date LIKE ? '''
    

    contents = []
    for row in rdd.collect():
        contents.append((row[0][0], row[0][1], row[1], row[0][0], row[0][1]))
    
    try:
        cursor.executemany(insert_records, contents)
        connection.commit()
    
    except sqlite3.Error as error:
        print({error})
    finally:
        cursor.close()
        connection.close()

def update_offset_table():
    global OFFSET
    
    connection = sqlite3.connect(os.path.join(os.getcwd(), f'../Database/cache.sqlite'))
    cursor = connection.cursor()

    query = f"UPDATE OFFSET_FINDER SET offsetval = {OFFSET} WHERE topic LIKE ?";
    cursor.execute(query, [TOPIC]);
    connection.commit();
    
    query = f"SELECT offsetval FROM OFFSET_FINDER WHERE topic LIKE ?"
    rows = cursor.execute(query, [TOPIC]).fetchall()

    if rows:
        OFFSET = rows[0][0]
    else:
        OFFSET = -1
        
    print({f"Updated Starting Offset for {TOPIC}": OFFSET})

    cursor.close()
    connection.close()
    
    raise StopExecution
        
def consumer_call():
    LIMIT_COUNT = 1000
    global TABLE_COUNT, OFFSET
    TABLE_COUNT = 1
    # delete_test_table()
    
    while True:
        query = init_df_table()
        sleep(10)
        
        value = spark.sql(f"SELECT * FROM {INMEMORY_TABLENAME}{TABLE_COUNT}").collect()
        spark.sql(f"DROP TABLE {INMEMORY_TABLENAME}{TABLE_COUNT}")
        
        TABLE_COUNT = (TABLE_COUNT+1)
        OFFSET += len(value)
        
        total_stock_count = len(value)
        
        print({"Stocks-data collected from select query": total_stock_count})
        
        if(total_stock_count == 0):
            update_offset_table()
        
        iter_count = 0
        
        while len(value):
            
            tweet_dict_list = []
            
            p_bar = tqdm(enumerate(value[:LIMIT_COUNT]))
            
            for indx, row in p_bar:
                jsonCopy = json.loads(row["Data"])
                jsonCopy['open'] = float(jsonCopy['open'])
                jsonCopy['close'] = float(jsonCopy['close'])
                jsonCopy['percentage'] = my_round1(jsonCopy['open'], jsonCopy['close'])
                tweet_dict_list.append(jsonCopy)
                p_bar.set_description(f'Working on "{indx + iter_count*LIMIT_COUNT + 1}/{total_stock_count}"')
                
            print({"Number of stock records" : len(tweet_dict_list)})
            print("----------------------------------------------------------------")
            query.awaitTermination(1)

            rdd = sc.parallelize(tweet_dict_list)

            newrdd = rdd.map(lambda row: (row['category'], row['stockDate'], row['percentage']))
            newrdd.collect()
            
            nextrdd = newrdd.map(lambda tup: ((tup[0], tup[1]), tup[2])).reduceByKey(lambda a, b: (a+b)/2).map(lambda row: (row[0], row[1]))
            nextrdd.collect()
            
            write_to_db(nextrdd)

            for i in range(LIMIT_COUNT):
                if(value):
                    value.pop(0)
            
            iter_count += 1
        
        

In [26]:
consumer_call()

root
 |-- Data: string (nullable = true)

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|         |stockdata1|       true|
+---------+----------+-----------+

{'Stocks-data collected from select query': 0}
{'Updated Starting Offset for gaming-stocks': 501}


/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=63, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 35864), raddr=('127.0.0.1', 34497)>
  self._sock = None


FOR reruns of the program with offset > 0,
cell 19 - 24 (cell that takes limited data from INMENORY TABLE, till sqlite3 db connection) - comment out fully, 
cell 25, dont call delete_test_table()
